In [95]:
import pandas as pd
from sqlalchemy import create_engine
import os
import shutil
import yaml
import re

In [124]:
def replace_simple_tags(txt, bb_uid):
    tags = {
        'u' : 'U',
        'b' : 'STRONG',
        'i' : 'EM',
        's' : 'STRIKE',
        'sup' : 'SUP',
        'sub' : 'SUB',
        'code' : 'PRE',
        '*' : 'LI'
    }
    for t_before, t_after in  tags.items():

        start_tag_before = f'[{t_before}:{bb_uid}]'
        start_tag_after = f'<{t_after}>'
        end_tag_before = f'[/{t_before}:{bb_uid}]'
        end_tag_before_m = f'[/{t_before}:m:{bb_uid}]'
        end_tag_before_u = f'[/{t_before}:u:{bb_uid}]'
        end_tag_before_o = f'[/{t_before}:o:{bb_uid}]'
        end_tag_after = f'</{t_after}>'
        
        txt = txt \
            .replace(start_tag_before, start_tag_after) \
            .replace(end_tag_before, end_tag_after) \
            .replace(end_tag_before_m, end_tag_after) \
            .replace(end_tag_before_u, end_tag_after) \
            .replace(end_tag_before_o, end_tag_after)
            
    return txt


In [215]:
def replace_linebreaks(txt, bb_uid):
    # first, remove excess lien breaks after selected end-tags
    rep1 =  r'\1' + f'[br:{bb_uid}]'
    txt = re.sub(f'(\[\/(list|quote|code|\*)(\:(m|o|u))?\:{bb_uid}\])\n', rep1, txt)
    # then, change all other lien breaks to BRs
    txt = re.sub('(?<!\n)\n(?!\n)', '<br />\n', txt)
    txt = re.sub('(?<!\n)\n\n+(?!\n)', '<br />\n<br />\n', txt)
    # then restore the linebreaks after selected tags for readability
    txt = re.sub(f'\[br\:{bb_uid}]', '\n', txt)
    return txt


In [216]:
print(replace_linebreaks(post_text, post_uid))

Drodzy Forumowicze i Forumowiczki!<br />
<br />
Były górki, były dołki, ale jak by na to nie patrzeć PWGay istnieje już od ponad 10 lat. I wyrażę chyba zdanie wszystkich osób zaangażowanych w jego tworzenie, jeżeli napiszę, że chcemy aby trwało i rozwijało się dalej.<br />
<br />
Na przestrzeni tych 10 lat już bardzo wiele osób pełniło funkcje administratorów i moderatorów forum – wielu z nich możecie już, szczególnie nowi użytkownicy, nie pamiętać. Obecna ekipa, mimo najszczerszych chęci też kiedyś odejdzie. Szczególnie, że PWG to forum studenckie, a żaden z obecnych modów i adminów już studentem nie jest.<br />
<br />
Dlatego też ogłaszamy wszem i wobec nabór na [b:1fhrcv7w]nowego administratora[/b:1fhrcv7w] PWGay.<br />
<br />
Nie jest to na pewno klasyczne ogłoszenie o pracę, ale pewne rozwiązania się jednak sprawdzają, także [i:1fhrcv7w]ad rem[/i:1fhrcv7w]:<br />
<br />
[size=150:1fhrcv7w][b:1fhrcv7w]Administrator PWGay[/b:1fhrcv7w][/size:1fhrcv7w]<br />
<br />
[u:1fhrcv7w]Zadania

In [158]:
def replace_quotes(txt, bb_uid):

    txt = re.sub(
        f'\[quote\=([^\:]+)\:{bb_uid}\]',
        r'<blockquote><p>\n»\1« pisze:<br />\n',
        txt
    )

    txt = re.sub(
        f'\[quote\:{uid}\]',
        r'<blockquote><p>\ncytat:<br />\n',
        txt
    )

    txt = re.sub(
        f'\[\/quote(\:(o|u|m))?\:{uid}\]',
        '\n</p></blockquote>',
        txt
    )

    return txt
    

In [44]:
db = os.environ.get('PSQL_DATABASE')
host = 'localhost'
user ='user'
port = '5432'
psql_engine = f'postgresql://{user}:@{host}:{port}/{db}'

In [45]:
with open("get_single_post.sql", "r") as f:
    sql_template = f.readlines()

In [102]:
post_id = 39034
post_id = 34524
slq_statement = ''.join(sql_template).format(post_id = post_id)
df = pd.read_sql(slq_statement, psql_engine)
post_content = df.loc[0].to_dict()

In [103]:
post_meta = {k : v for k, v in post_content.items() if k not in ['post_text', 'bbcode_uid']}
post_text = post_content['post_text']
post_uid = post_content['bbcode_uid']

In [169]:
print(post_text)

Drodzy Forumowicze i Forumowiczki!

Były górki, były dołki, ale jak by na to nie patrzeć PWGay istnieje już od ponad 10 lat. I wyrażę chyba zdanie wszystkich osób zaangażowanych w jego tworzenie, jeżeli napiszę, że chcemy aby trwało i rozwijało się dalej.

Na przestrzeni tych 10 lat już bardzo wiele osób pełniło funkcje administratorów i moderatorów forum – wielu z nich możecie już, szczególnie nowi użytkownicy, nie pamiętać. Obecna ekipa, mimo najszczerszych chęci też kiedyś odejdzie. Szczególnie, że PWG to forum studenckie, a żaden z obecnych modów i adminów już studentem nie jest.

Dlatego też ogłaszamy wszem i wobec nabór na [b:1fhrcv7w]nowego administratora[/b:1fhrcv7w] PWGay.

Nie jest to na pewno klasyczne ogłoszenie o pracę, ale pewne rozwiązania się jednak sprawdzają, także [i:1fhrcv7w]ad rem[/i:1fhrcv7w]:

[size=150:1fhrcv7w][b:1fhrcv7w]Administrator PWGay[/b:1fhrcv7w][/size:1fhrcv7w]

[u:1fhrcv7w]Zadania[/u:1fhrcv7w]:
[list:1fhrcv7w]
[*:1fhrcv7w]Bieżąca opieka nad Forum i Po

In [168]:
final_text = replace_linebreaks(post_text, post_uid)
final_text = replace_simple_tags(final_text, post_uid)
final_text = replace_quotes(final_text, post_uid)
print(final_text)

Drodzy Forumowicze i Forumowiczki!<br />
<br />
Były górki, były dołki, ale jak by na to nie patrzeć PWGay istnieje już od ponad 10 lat. I wyrażę chyba zdanie wszystkich osób zaangażowanych w jego tworzenie, jeżeli napiszę, że chcemy aby trwało i rozwijało się dalej.<br />
<br />
Na przestrzeni tych 10 lat już bardzo wiele osób pełniło funkcje administratorów i moderatorów forum – wielu z nich możecie już, szczególnie nowi użytkownicy, nie pamiętać. Obecna ekipa, mimo najszczerszych chęci też kiedyś odejdzie. Szczególnie, że PWG to forum studenckie, a żaden z obecnych modów i adminów już studentem nie jest.<br />
<br />
Dlatego też ogłaszamy wszem i wobec nabór na <STRONG>nowego administratora</STRONG> PWGay.<br />
<br />
Nie jest to na pewno klasyczne ogłoszenie o pracę, ale pewne rozwiązania się jednak sprawdzają, także <EM>ad rem</EM>:<br />
<br />
[size=150:1fhrcv7w]<STRONG>Administrator PWGay</STRONG>[/size:1fhrcv7w]<br />
<br />
<U>Zadania</U>:
[list:1fhrcv7w]
<LI>Bieżąca opieka 